<a href="https://colab.research.google.com/github/tusiime22/public_stuff/blob/main/Building_a_Production_AI_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Building a Production AI Application with Tecton
---

##### 💡 **NOT YET A TECTON USER?**

Sign-up at [explore.tecton.ai](https://explore.tecton.ai) for a free account that lets you try out this tutorial and explore Tecton's Web UI.

---

Tecton helps you build and productionize real-time ML models by making it easy
to define, test, and deploy features for training and serving.

Let’s see how quickly we can build a real-time fraud detection model and bring
it online.

In this tutorial we will:

1. Connect to data on S3
2. Define and test features
3. Generate a training dataset and train a model
4. Productionize our features for real-time serving
5. Run real-time inference to predict fraudulent transactions

This tutorial is expected to take about 30 minutes (record time for building a
real-time ML application 😎).

---

##### 💡 **TIP**

Most of this tutorial is intended to be run in a notebook. Some steps will explicitly note to run commands in your terminal.

---

## ⚙️ Install Pre-Reqs

First things first, let's install the Tecton SDK and other libraries used in this tutorial by running the cell below.

**NOTE**: When the `pip install` is finished you will need to restart the session in order to continue. Click "Restart session" in the dialog box or in the notebook cell.


In [1]:
!pip install 'tecton[rift]==1.1.0' gcsfs s3fs scikit-learn --force

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.1 MB/s eta 0:00:00
  Using cached setuptools-80.8.0-py3-none-any.whl.metadata (6.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.3 MB/s eta 0:00:00
INFO: pip is looking a

## ✅ Log in to Tecton

Next we will authenticate with your organization's Tecton account.

For users that just signed up via `explore.tecton.ai` you can leave this step as is. If your organization has its own Tecton account, replace `explore.tecton.ai` with your account url.

*Note: You need to press `enter` after pasting in your authentication code.*

In [1]:
import tecton

tecton.login('explore.tecton.ai') # replace with your URL

Please visit the following link to login and access the authentication code:
https://login.tecton.ai/oauth2/default/v1/authorize?response_type=code&client_id=0oazt04n72Fkm3APE357&redirect_uri=https%3A%2F%2Fwww.tecton.ai%2Fauthorization-callback&state=14405986852945169319&scope=openid+offline_access+profile+email&code_challenge_method=S256&code_challenge=ZNF4s2pt_X0hkU5AFnoujv2ycVrX3hAB0AvkcqpK064
Paste authentication code here and press [Enter]:rHq6MpmIYWvm7x6HnhUVwD4CZRd2WNrNzkN1-tAPrOo
✅ Authentication successful!


Let's then run some basic imports and setup that we will use later in the tutorial.

In [17]:
import pandas as pd

transactions_df = pd.read_parquet("s3://tecton.ai.public/tutorials/transactions.pq", storage_options={"anon": True})

display(transactions_df.head(50000))

,timestamp,user_id,transaction_id,merchant,merch_lat,merch_long,amount
0,2020-01-01 00:27:59.442071,user_1990251765,e8bd834c108c0929cd757f79e65acca0,Whole Foods,-45.676907,-171.601424,28.86
1,2020-01-01 01:11:01.384867,user_1284832379,0be03a2b9370b66073f0ed655423fa00,A&F,-73.685249,66.545372,20.46
2,2020-01-01 01:20:59.084788,user_9979340926,986eddaed2f4971b7d4d955dce678e28,Shaw's,84.224302,151.194256,98.39
3,2020-01-01 01:22:08.889972,user_8096819426,f75d2872395660da185b4c82912ffb4a,Zales,88.399805,-171.109611,97.74
4,2020-01-01 01:49:01.356931,user_8096819426,6112629434a2675e299b083cf96bfa7d,Quality Stores,-56.688061,-68.362769,27.53
...,...,...,...,...,...,...,...
49995,2022-09-26 19:24:52.464758,user_4133774204,6cc6faf22e2118183311145320a64c36,Carson Pirie Scott,-52.803581,-77.295152,49.39
49996,2022-09-26 19:48:11.515702,user_7306585851,1893f0f2599426f0326a6bb3614c3e84,Outback Steakhouse,-65.854467,42.024442,3.70
49997,2022-09-26 20:42:48.294086,user_3407770076,6d8c192742be750c3c0fef3d2871ab4d,Bojangles,-84.924616,92.799524,18.99
49998,2022-09-26 21:05:59.563039,user_9704575201,7629c5bc3a3b0bd7e6d7ea63af67c60b,C.R. Anthony,-67.559146,174.926897,125.85


from matplotlib import pyplot as plt
_df_57['merch_lat'].plot(kind='hist', bins=20, title='merch_lat')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_58['merch_long'].plot(kind='hist', bins=20, title='merch_long')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_59['amount'].plot(kind='hist', bins=20, title='amount')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_60.plot(kind='scatter', x='merch_lat', y='merch_long', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['timestamp']
  ys = series['amount']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_64.sort_values('timestamp', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('timestamp')
_ = plt.ylabel('amount')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['timestamp']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'timestamp'}, axis=1)
              .sort_values('timestamp', ascending=True))
  xs = counted['timestamp']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_65.sort_values('timestamp', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('timestamp')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_66['merch_lat'].plot(kind='line', figsize=(8, 4), title='merch_lat')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_67['merch_long'].plot(kind='line', figsize=(8, 4), title='merch_long')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_68['amount'].plot(kind='line', figsize=(8, 4), title='amount')
plt.gca().spines[['top', 'right']].set_visible(False)

In [3]:
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregate
from tecton.types import Field, String, Timestamp, Float64
from datetime import datetime, timedelta

tecton.conf.set("TECTON_OFFLINE_RETRIEVAL_COMPUTE_MODE", "rift")

Now we're ready to build!

## 🔎 Examine raw data

First let's examine some historical transaction data that we have available on
S3.

## 👩‍💻 Define and test features locally

In our data, we see that there's information on users' transactions over time.

Let's use this data to create the following features:

- A user's average transaction amount over 1, 3, and 7 days.
- A user's total transaction count over 1, 3, and 7 days.

To build these features, we will define a "Batch Source" and "Batch Feature
View" using Tecton's Feature Engineering Framework.

A Feature View is how we define our feature logic and give Tecton the
information it needs to productionize, monitor, and manage features.

Tecton's [development workflow](https://docs.tecton.ai/docs/the-feature-development-workflow) allows you
to build and test features, as well as generate training data entirely in a
notebook! Let's try it out.

In [21]:
transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/transactions.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)

# An entity defines the concept we are modeling features for
# The join keys will be used to aggregate, join, and retrieve features
user = Entity(name="user", join_keys=[Field("user_id", String)])

# We use Pandas to transform the raw data and Tecton aggregations to efficiently and accurately compute metrics across raw events
# Feature View decorators contain a wide range of parameters for materializing, cataloging, and monitoring features
@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="pandas",
    aggregation_interval=timedelta(days=1),
    features=[
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=1)),
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=3)),
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=7)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=1)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=3)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=7)),
    ],
    timestamp_field="timestamp",
)
def user_transaction_metrics(transactions):
    return transactions[["user_id", "timestamp", "amount"]]

## 🧪 Test features interactively

Now that we've defined our Feature View, we can use
`get_features_in_range` to produce a range of feature values and check out the
data.

In [22]:
start = datetime(2022, 1, 1)
end = datetime(2022, 2, 1)

df = user_transaction_metrics.get_features_in_range(start_time=start, end_time=end).to_pandas()

display(df.head(5000))

Credentials is None, retrying...
Credentials is None, retrying...
Credentials is None, retrying...
Credentials is None, retrying...
Credentials is None, retrying...
Credentials is None, retrying...


,user_id,amount_mean_1d_1d,amount_mean_3d_1d,amount_mean_7d_1d,amount_count_1d_1d,amount_count_3d_1d,amount_count_7d_1d,_valid_from,_valid_to
0,user_1284832379,1.750,285.3300,177.142857,1,4,7,2022-01-01 00:00:00+00:00,2022-01-02 00:00:00+00:00
1,user_1939957235,468.810,468.8100,175.191667,2,2,6,2022-01-03 00:00:00+00:00,2022-01-04 00:00:00+00:00
2,user_1997016327,254.475,123.1080,98.161000,2,5,10,2022-01-25 00:00:00+00:00,2022-01-26 00:00:00+00:00
3,user_2016327975,47.010,49.1400,142.374000,2,5,10,2022-01-07 00:00:00+00:00,2022-01-08 00:00:00+00:00
4,user_2121305484,40.110,88.9760,133.082222,1,5,9,2022-01-16 00:00:00+00:00,2022-01-17 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...
1524,user_9874666538,NaN,298.2525,387.665556,0,4,9,2022-01-16 00:00:00+00:00,2022-01-17 00:00:00+00:00
1525,user_3942331269,NaN,NaN,422.885000,0,0,2,2022-01-10 00:00:00+00:00,2022-01-12 00:00:00+00:00
1526,user_4183465914,NaN,NaN,422.200000,0,0,4,2022-01-02 00:00:00+00:00,2022-01-03 00:00:00+00:00
1527,user_6307583010,NaN,NaN,204.083333,0,0,3,2022-01-26 00:00:00+00:00,2022-01-27 00:00:00+00:00


## 🧮 Generate training data

We'll build our training dataset from labeled historical transactions and try to
predict the "is_fraud" column for a given transaction.

First, let's load our label dataset, which indicates whether a transaction in our historical dataset was fraudulent.

In [24]:
training_labels = pd.read_parquet("s3://tecton.ai.public/tutorials/labels.pq", storage_options={"anon": True})
display(training_labels.head(5000))

,transaction_id,is_fraud
0,e8bd834c108c0929cd757f79e65acca0,0
1,0be03a2b9370b66073f0ed655423fa00,0
2,986eddaed2f4971b7d4d955dce678e28,0
3,f75d2872395660da185b4c82912ffb4a,0
4,6112629434a2675e299b083cf96bfa7d,0
...,...,...
4995,c210d8fc45d78672e0328b7b1f31219f,0
4996,3cd296502fde3e51d25b01986a23691b,0
4997,e8544907a17265af0d84701a5423a9b7,0
4998,d3e10a7481af1e38360f19bca40d160f,0


Let's join our transactions dataset to our label dataset (on the `transaction_id` column) to produce a set of training events we'll then use to generate our training data.

In [26]:
training_events = training_labels.merge(transactions_df, on=['transaction_id'], how='left')[['user_id', 'timestamp', 'amount', 'is_fraud']]
display(training_events.head(5000))

,user_id,timestamp,amount,is_fraud
0,user_1990251765,2020-01-01 00:27:59.442071,28.86,0
1,user_1284832379,2020-01-01 01:11:01.384867,20.46,0
2,user_9979340926,2020-01-01 01:20:59.084788,98.39,0
3,user_8096819426,2020-01-01 01:22:08.889972,97.74,0
4,user_8096819426,2020-01-01 01:49:01.356931,27.53,0
...,...,...,...,...
4995,user_7145383296,2020-04-01 18:24:11.777127,70.96,0
4996,user_2121305484,2020-04-01 18:54:43.028430,147.49,0
4997,user_6348117987,2020-04-01 19:44:22.363085,89.16,0
4998,user_8816492034,2020-04-01 20:10:03.370543,956.88,0


Next, let's ask Tecton to join the features we just created into our labeled
events. Tecton will perform a
[time travel join](https://docs.tecton.ai/docs/reading-feature-data/reading-feature-data-for-training/constructing-training-data#a-note-on-point-in-time-correctness)
to fetch point-in-time correct feature values.

To do this we will create a "Feature Service" which defines the list of features
that will be used by our model.

We can call `get_features_for_events(training_events)` on the Feature Service to
get historically accurate features for each event.


In [27]:
from tecton import FeatureService

fraud_detection_feature_service = FeatureService(
    name="fraud_detection_feature_service", features=[user_transaction_metrics]
)

training_data = fraud_detection_feature_service.get_features_for_events(training_events).to_pandas().fillna(0)
display(training_data.sample(5))

Credentials is None, retrying...
Credentials is None, retrying...
Credentials is None, retrying...


,user_id,timestamp,amount,is_fraud,user_transaction_metrics__amount_count_1d_1d,user_transaction_metrics__amount_count_3d_1d,user_transaction_metrics__amount_count_7d_1d,user_transaction_metrics__amount_mean_1d_1d,user_transaction_metrics__amount_mean_3d_1d,user_transaction_metrics__amount_mean_7d_1d
34431,user_4400194105,2021-11-18 10:28:51.068957+00:00,1.52,0,2,4,8,47.155,28.1975,51.1225
86281,user_7145383296,2022-04-29 23:22:35.075465+00:00,11.82,0,2,2,5,87.990,87.9900,103.7820
63852,user_4063572189,2022-08-30 18:21:01.590086+00:00,76.39,0,0,1,1,0.000,924.3800,924.3800
46067,user_8468871048,2024-03-14 17:21:41.276057+00:00,11.77,0,1,1,6,9.340,9.3400,72.1500
4258,user_5638922576,2020-12-15 01:20:04.373847+00:00,22.69,0,0,2,4,0.000,5.8450,17.2500


## 🧠 Train a model

Once we have our training data set from Tecton, we can use whatever framework we
want for training the model.

In the example below, we'll train a simple Logistic Regression model using
sklearn!

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

df = training_data.drop(['user_id', 'timestamp', 'amount'], axis=1)

X = df.drop("is_fraud", axis=1)
y = df["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_cols = X_train.select_dtypes(exclude=["object"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

num_pipe = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="N/A"), OneHotEncoder(handle_unknown="ignore", sparse_output=False)
)

full_pipe = ColumnTransformer([("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)])

model = make_pipeline(full_pipe, LogisticRegression(max_iter=1000, random_state=42))

model.fit(X_train, y_train)

y_predict = model.predict(X_test)

print(metrics.classification_report(y_test, y_predict, zero_division=0))

Of course, you can continue building iterating on features and retraining your
model until you are ready to productionize.

## 🚀 Apply your Tecton application to production

Tecton objects get registered via a declarative workflow. Features are defined
as code in a repo and applied to a [workspace](https://docs.tecton.ai/docs/beta/introduction/tecton-concepts#workspace) in a Tecton account using the
Tecton CLI. A workspace is like a project for your team or org and corresponds to a single feature repository.

This declarative workflow enables productionisation best practices such as
"features as code," CI/CD, and unit testing.

---

##### ℹ️ **HEADS UP!**

This section requires your organization to have its own Tecton account. But don't fret! If you are a user of 'explore.tecton.ai', we've done these steps for you. You can read through it and continue with the rest of the tutorial, picking back up at the "Check on backfilling status" section below.

If you want to productionize your own features with your own data, you can sign up for an unrestricted free trial at [tecton.ai/free-trial](https://tecton.ai/free-trial).

---

### 1. Create a Tecton Feature Repository

Let's switch over from our notebook to a terminal and create a new Tecton
Feature Repository. For now we will put all our definitions in a single file.

✅ Run these commands to create a new Tecton repo.

```bash
mkdir tecton-feature-repo
cd tecton-feature-repo
touch features.py
tecton init
```

### 2. Fill in features.py and enable materialization

✅ Now copy & paste the definition of the Tecton objects you created in your
notebook to `features.py` (copied below).

On our Feature View we've added four parameters to enable backfilling and
ongoing materialization to the online and offline Feature Store:

- `online=True`
- `offline=True`
- `feature_start_time=datetime(2020,1,1)`
- `batch_schedule=timedelta(days=1)`

The offline and online Feature Stores are used for storing and serving feature values for training and inference. For more information, check out [Tecton Concepts](https://docs.tecton.ai/docs/beta/introduction/tecton-concepts#offline-feature-store).

When we apply our changes to a [Live Workspace](https://docs.tecton.ai/docs/beta/introduction/tecton-concepts#workspace), Tecton will automatically kick
off jobs to backfill feature data from `feature_start_time`. Frontfill jobs will
then run on the defined `batch_schedule`.

---

##### ℹ️ **INFO**

Besides the new materialization parameters, the code below is exactly the same as our definitions above. No changes are required when moving from interactive development to productionization!

---


**features.py**

```python
from tecton import Entity, BatchSource, FileConfig, batch_feature_view, Aggregate, FeatureService
from tecton.types import Field, String, Timestamp, Float64
from datetime import datetime, timedelta


transactions = BatchSource(
    name="transactions",
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/transactions.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
)

# An entity defines the concept we are modeling features for
# The join keys will be used to aggregate, join, and retrieve features
user = Entity(name="user", join_keys=[Field("user_id", String)])

# We use Pandas to transform the raw data and Tecton aggregations to efficiently and accurately compute metrics across raw events
# Feature View decorators contain a wide range of parameters for materializing, cataloging, and monitoring features
@batch_feature_view(
    description="User transaction metrics over 1, 3 and 7 days",
    sources=[transactions],
    entities=[user],
    mode="pandas",
    aggregation_interval=timedelta(days=1),
    timestamp_field="timestamp",
    features=[
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=1)),
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=3)),
        Aggregate(function="mean", input_column=Field("amount", Float64), time_window=timedelta(days=7)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=1)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=3)),
        Aggregate(function="count", input_column=Field("amount", Float64), time_window=timedelta(days=7)),
    ],
    online=True,
    offline=True,
    feature_start_time=datetime(2020, 1, 1),
    batch_schedule=timedelta(days=1),
)
def user_transaction_metrics(transactions):
    return transactions[["user_id", "timestamp", "amount"]]

fraud_detection_feature_service = FeatureService(
    name="fraud_detection_feature_service", features=[user_transaction_metrics]
)
```

### 3. Apply your changes to a new workspace

Our last step is to login to your organization's Tecton account and apply our
repo to a workspace!

✅ Run the following commands in your terminal to create a workspace and apply
your changes:

```bash
tecton login [your-org-account-name].tecton.ai
tecton workspace create [your-name]-quickstart --live
tecton apply
```

```
Using workspace "[your-name]-quickstart" on cluster https://explore.tecton.ai
✅ Imported 1 Python module from the feature repository
✅ Imported 1 Python module from the feature repository
⚠️  Running Tests: No tests found.
✅ Collecting local feature declarations
✅ Performing server-side feature validation: Initializing.
 ↓↓↓↓↓↓↓↓↓↓↓↓ Plan Start ↓↓↓↓↓↓↓↓↓↓

  + Create Batch Data Source
    name:           transactions

  + Create Entity
    name:           user

  + Create Transformation
    name:           user_transaction_metrics
    description:    Trailing average transaction amount over 1, 3 and 7 days

  + Create Batch Feature View
    name:           user_transaction_metrics
    description:    Trailing average transaction amount over 1, 3 and 7 days
    materialization: 11 backfills, 1 recurring batch job
    > backfill:     10 Backfill jobs 2020-01-01 00:00:00 UTC to 2023-08-16 00:00:00 UTC writing to the Offline Store
                    1 Backfill job 2023-08-16 00:00:00 UTC to 2023-08-23 00:00:00 UTC writing to both the Online and Offline Store
    > incremental:  1 Recurring Batch job scheduled every 1 day writing to both the Online and Offline Store

  + Create Feature Service
    name:           fraud_detection_feature_service

 ↑↑↑↑↑↑↑↑↑↑↑↑ Plan End ↑↑↑↑↑↑↑↑↑↑↑↑
 Generated plan ID is 8d01ad78e3194a5dbd3f934f04d71564
 View your plan in the Web UI: https://explore.tecton.ai/app/[your-name]-quickstart/plan-summary/8d01ad78e3194a5dbd3f934f04d71564
 ⚠️  Objects in plan contain warnings.

Note: Updates to Feature Services may take up to 60 seconds to be propagated to the real-time feature-serving endpoint.
Note: This workspace ([your-name]-quickstart) is a "Live" workspace. Applying this plan may result in new materialization jobs which will incur costs. Carefully examine the plan output before applying changes.
Are you sure you want to apply this plan to: "[your-name]-quickstart"? [y/N]> y
🎉 all done!
```

## 🟢 Check on backfilling status

Now that we've applied our features to a live workspace and enabled
materialization to the online and offline store, we can check on the status of
backfill jobs in the Tecton Web UI.

This can be found at the following URL (replace `[your-org-account-name]` and `[your-workspace-name]` with the appropriate values):

[https://[your-org-account-name].tecton.ai/app/repo/[your-workspace-name]/features/user_transaction_metrics/materialization](https://[your-org-account-name].tecton.ai/app/repo/[your-workspace-name]/features/user_transaction_metrics/materialization)

If you are using `explore.tecton.ai`, the URL will be:
[https://explore.tecton.ai/app/repo/prod/features/user_transaction_metrics/materialization](https://explore.tecton.ai/app/repo/prod/features/user_transaction_metrics/materialization)

Once the backfill jobs have completed, we can fetch feature values online!

## ⚡️ Create a function to retrieve features from Tecton's HTTP API

Now let's use Tecton's HTTP API to retrieve features at low latency.

To do this, you will first need to create a new Service Account and give it
access to read features from your workspace.

✅ Head to the following URL to create a new service account (replace "explore" with your organization's account name in the URL as necessary). Be sure to save the API key!

[https://explore.tecton.ai/app/settings/accounts-and-access/service-accounts?create-service-account=true](https://explore.tecton.ai/app/settings/accounts-and-access/service-accounts?create-service-account=true)

✅ If you are using `explore.tecton.ai`, this account will automatically be given the necessary privileges to read features from the "prod" workspace. Otherwise, you should give the service account access to read features from your newly created workspace by following these steps:

1. Navigate to the Service Account page by clicking on your new service account in the list at the URL above
2. Click on "Assign Workspace Access"
3. Select your workspace and give the service account the "Consumer" role

✅ Copy the generated API key into the code snippet below where it says `your-api-key`. Also be sure to replace the workspace and account name with your newly created workspace name and account name if necessary.

---

##### 🗒️ **NOTE**

Service account permissions may take a few minutes to update.

---

In [ ]:
import requests


def get_online_feature_data(user_id):
    TECTON_API_KEY = "your-api-key"  # replace with your API key
    WORKSPACE_NAME = "prod" # replace with your new workspace name if needed
    ACCOUNT_URL = "explore.tecton.ai" # replace with your org account URL if needed

    headers = {"Authorization": f"Tecton-key {TECTON_API_KEY}"}

    request_data = {
        "params": {
            "feature_service_name": "fraud_detection_feature_service",
            "join_key_map": {"user_id": user_id},
            "metadata_options": {"include_names": True},
            "workspace_name": WORKSPACE_NAME,
        }
    }

    url = f"https://{ACCOUNT_URL}/api/v1/feature-service/get-features"

    response = requests.post(url, json=request_data, headers=headers)
    return response.json()

Now we can use our function to retrieve features at low latency!

In [ ]:
user_id = "user_1990251765"

feature_data = get_online_feature_data(user_id)

if "error" in feature_data:
    print("ERROR:", feature_data["error"])
else:
    print(feature_data["result"])

## 💡 Create a function to make a prediction given feature data

Now that we can fetch feature data online, let's create a function that takes a
feature vector and runs model inference to get a fraud prediction.

---

##### ℹ️ **INFO**

Typically you'd instead use a model serving API that is hosting your model. Here we run inference directly in our notebook for simplicity.

---

In [ ]:
import pandas as pd


def get_prediction_from_model(feature_data):
    columns = [f["name"].replace(".", "__") for f in feature_data["metadata"]["features"]]
    data = [feature_data["result"]["features"]]

    features = pd.DataFrame(data, columns=columns)[X.columns]

    return model.predict(features)[0]

## ✨ Run inference using features from Tecton

Let's combine these functions and run inference!

We can fetch our online features from Tecton, call our inference function, and
get a prediction.

In [ ]:
user_id = "user_1990251765"

online_feature_data = get_online_feature_data(user_id)
prediction = get_prediction_from_model(online_feature_data)

print(prediction)

## 🔥 Create a function to evaluate a user transaction and accept or reject it

Our final step is to use our new fraud prediction pipeline to make decisions and
take action in our application.

In the function below we use simple business logic to decide whether to accept
or reject a transaction based on our predicted fraud score.

In [ ]:
def evaluate_transaction(user_id):
    online_feature_data = get_online_feature_data(user_id)
    is_predicted_fraud = get_prediction_from_model(online_feature_data)

    if is_predicted_fraud == 0:
        return "Transaction accepted."
    else:
        return "Transaction denied."

## 💰 Evaluate a transaction

Put it all together and we have a single online, low-latency decision API for
our application. Try it out below!

In [ ]:
evaluate_transaction("user_1990251765")

## ⭐️ Conclusion

In this tutorial, we were able to quickly make an end to end real-time fraud
detection application using features built in Tecton.

We tested our features, built training data sets, productionized features with
engineering best practices, retrieved features online, and made decisions in
real time!

But Tecton can do so much more:

- streaming features
- real-time features
- monitoring
- unit testing
- cataloging and discovery
- access controls
- cost management
- rules engines

...and more.

Next, we recommend checking out our tutorial on
[building streaming features](https://colab.research.google.com/github/tecton-ai/demo-notebooks/blob/main/Tutorial_Building_Streaming_Features_with_Tecton.ipynb) to learn more
about how to infuse your models with real-time data using nothing more than
Python!